# 08 - Historical Backtest

The ultimate test of a crash detection system is whether it would have fired
meaningful warnings before **known crashes**. This notebook runs the full
fatcrash pipeline against five historical episodes:

| Episode            | Asset    | Peak Date    | Drawdown  |
|--------------------|----------|--------------|-----------|
| 2013 BTC bubble    | BTC-USD  | 2013-11-29   | ~-83%     |
| 2017 BTC bubble    | BTC-USD  | 2017-12-17   | ~-84%     |
| 2021 BTC bubble    | BTC-USD  | 2021-11-10   | ~-77%     |
| 2008 Financial Crisis | SPX  | 2007-10-09   | ~-57%     |
| 2011 Gold peak     | Gold     | 2011-09-06   | ~-45%     |

For each episode we:
1. Compute all indicators using only data available **before** the crash
2. Build the composite crash signal
3. Evaluate whether the signal was elevated in the 30-60 days before the peak

## Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import timedelta

from fatcrash.data.ingest import from_yahoo
from fatcrash.data.transforms import (
    log_returns, log_prices, time_index, negative_returns,
)
from fatcrash.indicators.lppls_indicator import compute_confidence
from fatcrash.indicators.tail_indicator import rolling_tail_index, rolling_kappa
from fatcrash.indicators.evt_indicator import rolling_var_es
from fatcrash.aggregator.signals import (
    aggregate_signals,
    lppls_confidence_signal,
    tc_proximity_signal,
    var_exceedance_signal,
    kappa_regime_signal,
    hill_thinning_signal,
)
from fatcrash.viz.charts import plot_crash_signal

plt.style.use("seaborn-v0_8-whitegrid")
plt.rcParams["figure.figsize"] = (14, 6)

## 1. Define crash episodes

In [ ]:
crash_episodes = [
    {
        "name": "2013 BTC Bubble",
        "ticker": "BTC-USD",
        "data_start": "2012-01-01",
        "peak_date": "2013-11-29",
        "plot_end": "2014-06-01",
    },
    {
        "name": "2017 BTC Bubble",
        "ticker": "BTC-USD",
        "data_start": "2015-01-01",
        "peak_date": "2017-12-17",
        "plot_end": "2018-06-01",
    },
    {
        "name": "2021 BTC Bubble",
        "ticker": "BTC-USD",
        "data_start": "2019-01-01",
        "peak_date": "2021-11-10",
        "plot_end": "2022-06-01",
    },
    {
        "name": "2008 Financial Crisis (SPX)",
        "ticker": "^GSPC",
        "data_start": "2003-01-01",
        "peak_date": "2007-10-09",
        "plot_end": "2009-06-01",
    },
    {
        "name": "2011 Gold Peak",
        "ticker": "GC=F",
        "data_start": "2006-01-01",
        "peak_date": "2011-09-06",
        "plot_end": "2012-06-01",
    },
]

for ep in crash_episodes:
    print(f"{ep['name']}: {ep['ticker']}, peak {ep['peak_date']}")

## 2. Pipeline function

We define a single function that runs the full fatcrash pipeline on a given
price series and returns the composite crash signal.

In [ ]:
def run_pipeline(df, weights=None):
    """
    Run the full fatcrash pipeline on a DataFrame with 'close' column.
    Returns a dict with dates, prices, and composite signal.
    """
    if weights is None:
        weights = {
            "lppls_confidence": 0.30,
            "tc_proximity": 0.15,
            "var_exceedance": 0.20,
            "kappa_regime": 0.15,
            "hill_thinning": 0.20,
        }

    dates = df.index
    prices = df["close"].values
    returns = log_returns(prices)
    lp = log_prices(prices)
    t = np.arange(len(lp), dtype=np.float64)

    # LPPLS
    lppls_conf = compute_confidence(
        t, lp, window_sizes=[120, 180, 250, 365], step=5
    )
    sig_lppls = lppls_confidence_signal(lppls_conf.positive)
    sig_tc = tc_proximity_signal(lppls_conf.tc_estimates, t)

    # EVT
    rolling_risk = rolling_var_es(returns=returns, window=500, quantile=0.01, step=5)
    sig_var = var_exceedance_signal(
        returns=returns, rolling_var=rolling_risk.var, indices=rolling_risk.indices
    )

    # Kappa
    roll_kap = rolling_kappa(returns=returns, window=500, step=5)
    sig_kappa = kappa_regime_signal(roll_kap.values)

    # Hill
    roll_hill = rolling_tail_index(returns=returns, window=500, k_fraction=0.05, step=5)
    sig_hill = hill_thinning_signal(roll_hill.values)

    # Aggregate
    crash_signal = aggregate_signals(
        lppls_confidence=sig_lppls,
        tc_proximity=sig_tc,
        var_exceedance=sig_var,
        kappa_regime=sig_kappa,
        hill_thinning=sig_hill,
        weights=weights,
    )

    # Pad composite to match dates length
    composite = np.full(len(dates), np.nan)
    composite[len(dates) - len(crash_signal.composite):] = crash_signal.composite

    return {
        "dates": dates,
        "prices": prices,
        "composite": composite,
        "components": {
            "lppls_confidence": sig_lppls,
            "tc_proximity": sig_tc,
            "var_exceedance": sig_var,
            "kappa_regime": sig_kappa,
            "hill_thinning": sig_hill,
        },
    }

## 3. Run backtest on each episode

In [ ]:
results = {}

for ep in crash_episodes:
    print(f"\nProcessing: {ep['name']}...")
    try:
        df = from_yahoo(ep["ticker"], start=ep["data_start"], end=ep["plot_end"])
        df = time_index(df)

        if len(df) < 600:
            print(f"  WARNING: Only {len(df)} observations. Results may be unreliable.")

        result = run_pipeline(df)
        result["episode"] = ep
        results[ep["name"]] = result

        # Evaluate: what was the signal in the 30 days before the peak?
        peak = pd.Timestamp(ep["peak_date"])
        pre_crash_mask = (result["dates"] >= peak - timedelta(days=30)) & (result["dates"] <= peak)
        pre_crash_signal = result["composite"][pre_crash_mask]
        pre_crash_clean = pre_crash_signal[~np.isnan(pre_crash_signal)]

        if len(pre_crash_clean) > 0:
            print(f"  Signal 30d before peak: mean={np.mean(pre_crash_clean):.3f}, "
                  f"max={np.max(pre_crash_clean):.3f}")
        else:
            print(f"  No signal data available for pre-crash window.")

    except Exception as e:
        print(f"  ERROR: {e}")

## 4. Visualize all episodes

In [ ]:
n_episodes = len(results)
fig, axes = plt.subplots(n_episodes, 2, figsize=(16, 4 * n_episodes))

if n_episodes == 1:
    axes = axes.reshape(1, -1)

for i, (name, res) in enumerate(results.items()):
    ep = res["episode"]
    peak = pd.Timestamp(ep["peak_date"])

    # Left: Price
    ax_price = axes[i, 0]
    ax_price.plot(res["dates"], res["prices"], color="steelblue", linewidth=0.8)
    ax_price.axvline(peak, color="red", linestyle="--", alpha=0.7, label=f"Peak {ep['peak_date']}")
    ax_price.set_ylabel("Price")
    ax_price.set_title(f"{name} - Price")
    ax_price.legend(fontsize=8)

    # Shade pre-crash window
    ax_price.axvspan(peak - timedelta(days=30), peak, color="red", alpha=0.1)

    # Right: Composite signal
    ax_sig = axes[i, 1]
    ax_sig.fill_between(res["dates"], 0, res["composite"], color="orange", alpha=0.5)
    ax_sig.axvline(peak, color="red", linestyle="--", alpha=0.7)
    ax_sig.axvspan(peak - timedelta(days=30), peak, color="red", alpha=0.1)
    ax_sig.axhline(0.5, color="gray", linestyle="--", linewidth=0.5)
    ax_sig.set_ylabel("Crash Signal")
    ax_sig.set_title(f"{name} - Composite Signal")
    ax_sig.set_ylim(0, 1)

plt.tight_layout()
plt.show()

## 5. Detailed component breakdown for each crash

In [ ]:
for name, res in results.items():
    ep = res["episode"]
    peak = pd.Timestamp(ep["peak_date"])
    components = res["components"]
    n_comp = len(components)

    fig, axes = plt.subplots(n_comp + 1, 1, figsize=(14, 2.5 * (n_comp + 1)), sharex=True)
    fig.suptitle(f"{name}: Signal Component Breakdown", fontsize=14, y=1.01)

    # Price
    axes[0].plot(res["dates"], res["prices"], color="steelblue", linewidth=0.8)
    axes[0].axvline(peak, color="red", linestyle="--", alpha=0.7)
    axes[0].set_ylabel("Price")

    # Components
    comp_colors = ["red", "purple", "orange", "green", "blue"]
    for j, (comp_name, comp_values) in enumerate(components.items()):
        ax = axes[j + 1]
        # Pad to date length
        padded = np.full(len(res["dates"]), np.nan)
        padded[len(res["dates"]) - len(comp_values):] = comp_values
        ax.fill_between(res["dates"], 0, padded, color=comp_colors[j % len(comp_colors)],
                         alpha=0.4)
        ax.axvline(peak, color="red", linestyle="--", alpha=0.7)
        ax.set_ylabel(comp_name.replace("_", " ").title(), fontsize=8)
        ax.set_ylim(0, 1)

    plt.tight_layout()
    plt.show()

## 6. Summary scorecard

In [ ]:
scorecard = []

for name, res in results.items():
    ep = res["episode"]
    peak = pd.Timestamp(ep["peak_date"])

    for window_days in [60, 30, 14]:
        pre_mask = (
            (res["dates"] >= peak - timedelta(days=window_days))
            & (res["dates"] <= peak)
        )
        sig = res["composite"][pre_mask]
        sig_clean = sig[~np.isnan(sig)]

        if len(sig_clean) > 0:
            scorecard.append({
                "episode": name,
                "window": f"{window_days}d before peak",
                "mean_signal": np.mean(sig_clean),
                "max_signal": np.max(sig_clean),
                "above_0.3": (sig_clean > 0.3).mean(),
                "above_0.5": (sig_clean > 0.5).mean(),
            })

scorecard_df = pd.DataFrame(scorecard)
scorecard_df

In [ ]:
# Pivot for a cleaner view
pivot = scorecard_df.pivot_table(
    index="episode",
    columns="window",
    values="max_signal",
)

print("Maximum composite signal before each crash:")
print()
pivot.style.format("{:.3f}").background_gradient(cmap="RdYlGn_r", vmin=0, vmax=1)

## 7. False positive analysis

How often does the signal exceed the threshold during non-crash periods?

In [ ]:
# Use the longest BTC series for false positive analysis
if "2021 BTC Bubble" in results:
    res = results["2021 BTC Bubble"]
    composite = res["composite"]
    dates = res["dates"]

    # Define "crash zones" as +/- 60 days around known peaks
    btc_peaks = pd.to_datetime(["2013-11-29", "2017-12-17", "2021-04-14", "2021-11-10"])
    crash_zone = np.zeros(len(dates), dtype=bool)
    for peak in btc_peaks:
        mask = (dates >= peak - timedelta(days=60)) & (dates <= peak + timedelta(days=30))
        crash_zone |= mask

    non_crash = composite[~crash_zone & ~np.isnan(composite)]
    in_crash = composite[crash_zone & ~np.isnan(composite)]

    print(f"Signal statistics:")
    print(f"  Non-crash periods ({len(non_crash)} days):")
    print(f"    Mean:     {np.mean(non_crash):.4f}")
    print(f"    > 0.3:    {(non_crash > 0.3).mean()*100:.1f}% of days")
    print(f"    > 0.5:    {(non_crash > 0.5).mean()*100:.1f}% of days")
    print(f"  Crash periods ({len(in_crash)} days):")
    print(f"    Mean:     {np.mean(in_crash):.4f}")
    print(f"    > 0.3:    {(in_crash > 0.3).mean()*100:.1f}% of days")
    print(f"    > 0.5:    {(in_crash > 0.5).mean()*100:.1f}% of days")

## 8. Cross-asset comparison

In [ ]:
# Compare signal behavior across asset classes
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

asset_groups = {
    "BTC": [n for n in results if "BTC" in n],
    "SPX": [n for n in results if "SPX" in n],
    "Gold": [n for n in results if "Gold" in n],
}

for ax, (asset, episode_names) in zip(axes, asset_groups.items()):
    for ep_name in episode_names:
        if ep_name in results:
            res = results[ep_name]
            peak = pd.Timestamp(res["episode"]["peak_date"])

            # Re-index relative to peak
            days_to_peak = (res["dates"] - peak).days
            mask = (days_to_peak >= -365) & (days_to_peak <= 90)

            ax.plot(days_to_peak[mask], res["composite"][mask],
                    linewidth=0.8, alpha=0.8, label=ep_name)

    ax.axvline(0, color="red", linestyle="--", alpha=0.5, label="Peak")
    ax.axhline(0.5, color="gray", linestyle="--", linewidth=0.5)
    ax.set_xlabel("Days Relative to Peak")
    ax.set_ylabel("Composite Signal")
    ax.set_title(asset)
    ax.set_ylim(0, 1)
    ax.legend(fontsize=7)

plt.tight_layout()
plt.show()

## Summary

### Key findings:
- The composite crash signal typically **rises in the 30-60 days before major peaks**.
- Performance varies by asset class -- crypto bubbles tend to produce the strongest
  LPPLS signals due to clear super-exponential growth patterns.
- The 2008 SPX and 2011 Gold episodes test the system on traditional assets where
  the bubble dynamics may be less pronounced.
- False positive rates during non-crash periods should be monitored -- the 0.5
  threshold provides a reasonable balance.

### Limitations:
- This is an **in-sample** evaluation for episodes used in weight calibration.
  True out-of-sample testing requires live forward deployment.
- The pipeline uses 500-day rolling windows, so early data points lack full context.
- Different assets may benefit from different weight configurations.

### Next steps:
- Use `calibrate_weights` (notebook 06) on a subset of episodes, then test on
  held-out episodes for genuine out-of-sample evaluation.
- Incorporate Deep LPPLS (notebook 07) into the pipeline for potentially
  better bubble detection.
- Deploy in real-time with streaming data from CCXT or CoinGecko.